<div align="right" style="text-align:right"><a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Licença Creative Commons" style="border-width:0; float:right" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br><br><i>Prof. Marcelo de Souza</i><br>marcelo.desouza@udesc.br</div>

# Investidor [genérico]

Programação linear com Pyomo.

---

**Dados:**
+ $C$: capital
+ $r_i$: rendimento do investimento $i = \{1, \ldots, n\}$
+ $\text{m}^\prime_i$: percentual mínimo do investimento $i = \{1, \ldots, n\}$
+ $\text{m}^{\prime\prime}_i$: percentual máximo do investimento $i = \{1, \ldots, n\}$

**Variáveis de decisão:**
+ $x_i$: valor indicado para o investimento $i = \{1, \ldots, n\}$

**Com isso, temos o modelo:**

$$
\begin{align} 
  \text{maximiza}\quad  &\sum_{i=1}^{n} r_i x_i \\
  \text{sujeito a}\quad &\sum_{i=1}^{n} x_i = C \\
                        &x_j \ge m^\prime_j \left(\sum_{i=1}^n x_i\right),\quad &j = \{1, \ldots, n\} \\
                        &x_j \le m_j^{\prime\prime} \left(\sum_{i=1}^n x_i\right),\quad &j = \{1, \ldots, n\} \\
                        &x_i \ge 0,\quad &i = \{1, \ldots, n\}
\end{align} 
$$

---

Preparação para execução no Google Colab

In [ ]:
!pip install -qq pyomo
!apt-get install -y -qq glpk-utils

In [2]:
instance = {
    'n': 4,
    'r': [0.05, 0.02, 0.1, 0.04],
    'C': 23000,
    'm1': [0.21, 0.07, 0.14, 0.11],
    'm2': [0.53, 0.51, 0.42, 0.42]
}

In [7]:
from pyomo.environ import *

model = ConcreteModel()

model.x = Var(range(instance['n']), domain=NonNegativeReals)

model.obj = Objective(expr=sum(model.x[i] * instance['r'][i] for i in range(instance['n'])), sense=maximize)

model.constraints = ConstraintList()
model.constraints.add(sum(model.x[i] for i in range(instance['n'])) == instance['C'])

for j in range(instance['n']):
    model.constraints.add(model.x[j] >= instance['m1'][j] * sum(model.x[i] for i in range(instance['n'])))
    model.constraints.add(model.x[j] <= instance['m2'][j] * sum(model.x[i] for i in range(instance['n'])))
    
solver = SolverFactory('glpk')
results = solver.solve(model)

if results.solver.termination_condition == 'optimal':
    print('Investimentos:')
    for i in range(instance['n']):
        print(f'{i+1}: {model.x[i]()}')
    print(f'Rendimento: {model.obj()}')
else:
    print()

Investimentos:
1: 9200.0
2: 1610.0
3: 9660.0
4: 2530.0
Rendimento: 1559.4
